In [ ]:
import GPy, scipy
from gpmultipy import dataset

In [ ]:
%matplotlib inline

In [ ]:
%pylab

In [ ]:
def wellPosition(number):
    number = int(number)
    if number > 100:
        return wellPosition(number%100)
    if number == 0:
        return 9,9
    return [(number-1) % 10, (number-1)/10]

In [ ]:
GPy.kern.ChangePointBasisFuncKernel(1,0.5,variance=.1).K(xgp)

In [ ]:
ktemp = GPy.kern.RBF(1) * (GPy.kern.Bias(1) + GPy.kern.ChangePointBasisFuncKernel(1,0.5,variance=.1))
plt.imshow(ktemp.K(xgp))

In [ ]:
x

In [ ]:
GPy.inference.mcmc.HMC()

In [ ]:
# plt.plot(x,y,c='k',alpha=.4); plt.plot(x,y.mean(1),c='r',lw=3);
# plt.xlabel("time (normalized)",fontsize=25)
# plt.ylabel("OD (normalized)",fontsize=25)
# plt.xticks(np.arange(.2,1,.2),fontsize=20)
# plt.yticks(np.arange(-2,2,.5),fontsize=20)

# plt.savefig("figures/ura3-pq-replicate/data-singleReplicate.pdf",bbox_inches='tight',dpi=300)

In [ ]:
# plt.figure(figsize=(6,4))

# plt.plot(x,y-y.mean(1)[:,None],c='k',alpha=.8)

# plt.xlabel("time (normalized)",fontsize=25)
# plt.ylabel("OD (normalized)",fontsize=25)
# plt.xticks(fontsize=20)
# plt.yticks(fontsize=20)

In [ ]:
bios = ds.meta.Bio.unique()[1:]


# residuals
plt.figure(figsize=(5*4,bios.shape[0]/5*4))

for ind,b in zip(range(bios.shape[0]),bios):
    x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',Bio=b,effects=['Well'])

    position = np.array(map(wellPosition,labels[0]))
    dist = np.column_stack((position.min(1),(9-position).min(1))).min(1)
    select = dist != 0
    if not any(select):
        continue    
    y = y[:,select]

    y = (y-ymean)/ystd
    
    plt.subplot(len(bios)/5+1,5,ind+1)
    plt.title(b)
    
    #plt.plot(x,y-y.mean(1)[:,None],c='k',alpha=.8)
    plt.plot(x,y.std(1),c='k',alpha=.8)
    plt.ylim(0,.3)

    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    
plt.tight_layout()


#data
plt.figure(figsize=(5*4,bios.shape[0]/5*4))

for ind,b in zip(range(bios.shape[0]),bios):
    x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',Bio=b,effects=['Well'])

    position = np.array(map(wellPosition,labels[0]))
    dist = np.column_stack((position.min(1),(9-position).min(1))).min(1)
    select = dist != 0
    if not any(select):
        continue    
    y = y[:,select]

    y = (y-ymean)/ystd
    
    plt.subplot(len(bios)/5+1,5,ind+1)
    plt.title(b)
    
    plt.plot(x,y,c='k',alpha=.4); plt.plot(x,y.mean(1),c='r',lw=3);

    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    
plt.tight_layout()

In [ ]:
ds = dataset.DataSet('data/normalized/ura3-pq-replicate/')
x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',Bio='K',effects=['Well'])

position = np.array(map(wellPosition,labels[0]))
dist = np.column_stack((position.min(1),(9-position).min(1))).min(1)
select = dist != 0
y = y[:,select]

ymean,ystd = y.mean(), y.std()
y = (y-y.mean())/y.std()

In [ ]:
xgp = np.zeros((x.shape[0]*y.shape[1],2))
xgp[:,0] = np.tile(x[:,0],y.shape[1])
xgp[:,1] = np.repeat(range(y.shape[1]),x.shape[0])

ygp = y.ravel(1)[:,None]

In [ ]:
k = GPy.kern.Hierarchical([GPy.kern.RBF(1),GPy.kern.RBF(1)])
plt.imshow(k.K(xgp))

In [ ]:
m = GPy.models.GPRegression(xgp,ygp,k)

m

In [ ]:
m.optimize()
m

In [ ]:
xtest = np.zeros((x.shape[0]*y.shape[1],1+y.shape[1]))
xtest[:,0] = np.tile(x[:,0],y.shape[1])

for i in range(y.shape[1]):
    xtest[i*x.shape[0]:(i+1)*x.shape[0],1+i] = 1

ktest = GPy.kern.RBF(1) * GPy.kern.RBF(y.shape[1],active_dims=range(1,y.shape[1]+1), lengthscale=1)

plt.imshow(ktest.K(xtest),interpolation='none',cmap='Blues')

In [ ]:
plt.imshow(k.rbf_1.K(xgp))

In [ ]:
m.plot(fixed_inputs=[(1,-1)])

In [ ]:
xpred = np.zeros((x.shape[0],2))
xpred[:,0] = x[:,0]
xpred[:,1] = y.shape[1]

mu,cov = m.predict(xpred,full_cov=True)
samples = scipy.stats.multivariate_normal.rvs(mu[:,0],cov,size=10)
plt.plot(samples.T);

## Linear + RBF kernel 

In [ ]:
k = GPy.kern.Hierarchical([GPy.kern.RBF(1),GPy.kern.RBF(1)*GPy.kern.Linear(1)])
plt.imshow(k.K(xgp))

In [ ]:
m = GPy.models.GPRegression(xgp,ygp,k)

m

In [ ]:
m.optimize()
m

In [ ]:
plt.imshow(k.mul.K(xgp))

In [ ]:
xpred = np.zeros((x.shape[0],2))
xpred[:,0] = x[:,0]
xpred[:,1] = y.shape[1]

mu,cov = m.predict(xpred,full_cov=True)
samples = scipy.stats.multivariate_normal.rvs(mu[:,0],cov,size=20)
plt.plot(samples.T);

In [ ]:
plt.figure(figsize=(6,4))

plt.plot(x,y.std(1))
plt.plot(x,samples.std(0));
plt.plot(x,np.sqrt(k.mul.K(xpred).diagonal()));

plt.xlabel("time (normalized)",fontsize=25)

### More polynomials in linear term 

In [ ]:
p = 4

xgpPoly = np.zeros((x.shape[0]*y.shape[1],p+1))

for i in range(p):
    xgpPoly[:,i] = np.tile(x[:,0],y.shape[1])**(i+1)
    
xgpPoly[:,-1] = np.repeat(range(y.shape[1]),x.shape[0])

In [ ]:
plt.plot(xgpPoly[:,:-1])

In [ ]:
kfunc = GPy.kern.RBF(p,ARD=True)
kreplicate = GPy.kern.RBF(1)*(GPy.kern.Linear(p,ARD=True) + GPy.kern.Bias(1))

# k = GPy.kern.Hierarchical([GPy.kern.RBF(p,ARD=True),GPy.kern.RBF(1)*GPy.kern.Linear(p,ARD=True)]) 
k = GPy.kern.Hierarchical([kfunc,kreplicate]) 

# k.rbf.lengthscale = [0.150116321796, 1e-5, 1e-5]
# k.rbf.lengthscale.fix()

plt.imshow(k.K(xgpPoly))

In [ ]:
m = GPy.models.GPRegression(xgpPoly,ygp,k)

m

In [ ]:
m.optimize()
m

In [ ]:
k.mul.sum.linear.variances

In [ ]:
xpred = np.zeros((x.shape[0],p+1))

for i in range(p):
    xpred[:,i] = x[:,0]**(i+1)
xpred[:,-1] = y.shape[1]

mu,cov = m.predict(xpred,full_cov=True)
samples = scipy.stats.multivariate_normal.rvs(mu[:,0],cov,size=10)

In [ ]:
plt.figure(figsize=(10,6))
plt.subplot(121)
plt.plot(y)
plt.ylim(-2.2,2.0)

plt.subplot(122)
plt.plot(samples.T);
plt.ylim(-2.2,2.0)

In [ ]:
plt.figure(figsize=(6,4))

plt.plot(x,y.std(1))
# plt.plot(x,samples.std(0));

for b in ['F','G','H','I']:
    _,ytemp,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',Bio=b,effects=['Well'])
    position = np.array(map(wellPosition,labels[0]))
    dist = np.column_stack((position.min(1),(9-position).min(1))).min(1)
    select = dist != 0
    ytemp = ytemp[:,select]

    ytemp = (ytemp-ymean)/ystd
    plt.plot(x,ytemp.std(1),c='r',alpha=.7)

plt.xlabel("time (normalized)",fontsize=25)

In [ ]:
plt.figure(figsize=(6,4))

plt.plot(x,y.std(1))
plt.plot(x,np.sqrt(cov.diagonal()));

plt.xlabel("time (normalized)",fontsize=25)

In [ ]:
plt.plot(scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]), k.mul.K(xpred),size=10).T);
# plt.plot(scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]), ktemp.K(xpred),size=10).T);

In [ ]:
keffect = GPy.kern.RBF(1,variance=k.rbf.variance*1e-1,lengthscale=k.rbf.lengthscale[0])

nsample = 2
xeffect = np.zeros((x.shape[0]*nsample*2,p+1))

for i in range(p):
    xeffect[:,i] = np.tile(x[:,0],nsample*2)**(i+1)
    
xeffect[:,-1] = np.repeat(range(nsample*2),x.shape[0])

cov = keffect.K(xeffect)
# cov[xeffect.shape[0]/2:,:xeffect.shape[0]/2] = -cov[xeffect.shape[0]/2:,:xeffect.shape[0]/2]
# cov[:xeffect.shape[0]/2,xeffect.shape[0]/2:] = -cov[:xeffect.shape[0]/2,xeffect.shape[0]/2:]

cov += kreplicate.K(xeffect)
cov += np.eye(cov.shape[0])*m.likelihood.variance

plt.imshow(cov,interpolation='none')

In [ ]:
rv = scipy.stats.multivariate_normal(np.zeros(cov.shape[0]),cov)
sample = rv.rvs()
sample = sample.reshape((nsample*2,x.shape[0])).T

plt.plot(sample)

In [ ]:
op = np.zeros((x.shape[0], nsample*2*x.shape[0]))

for i in range(x.shape[0]):
    op[i,i:x.shape[0]*nsample:x.shape[0]] = 1
    op[i,i+x.shape[0]*nsample::x.shape[0]] = -1
    
plt.imshow(op,interpolation='none',aspect='auto')

In [ ]:
opCov = np.dot(op,np.dot(cov,op.T))
plt.imshow(opCov)

In [ ]:
opCov.diagonal()

In [ ]:
xpred = np.zeros((x.shape[0]*y.shape[1],p+1))

for i in range(p):
    xpred[:,i] = np.tile(x[:,0]**(i+1),y.shape[1])
xpred[:,-1] = np.repeat(range(y.shape[1],y.shape[1]*2),x.shape[0])

mu,cov = m.predict(xpred,full_cov=True)

In [ ]:
for i in range(10):

    plt.figure(figsize=(10,3))
    plt.subplot(131)
    plt.plot(y)
    plt.ylim(-2.2,2.0)

    plt.subplot(132)
    samples = scipy.stats.multivariate_normal.rvs(mu[:,0],cov,size=1)
    plt.plot(samples.reshape((y.shape[1],x.shape[0])).T);
    plt.ylim(-2.2,2.0)

    plt.subplot(133)
    plt.plot(x,y.std(1))
    plt.plot(x,samples.reshape((y.shape[1],x.shape[0])).T.std(1));

### Experimental Design 

In [ ]:
kfunc.lengthscale

In [ ]:
keffect = kfunc.copy()
keffect.variance = 1e-1*keffect.variance
keffect.lengthscale = np.array([1,100,100,100])
effect = scipy.stats.multivariate_normal.rvs(np.zeros(xpred.shape[0]), keffect.K(xpred))
plt.plot(effect)

In [ ]:
from GPy.kern.src.kern import CombinationKernel
from GPy.kern.src.independent_outputs import index_to_slices

In [ ]:
class Hierarchical(CombinationKernel):
    """
    A kernel which can represent a simple hierarchical model.
    See Hensman et al 2013, "Hierarchical Bayesian modelling of gene expression time
    series across irregularly sampled replicates and clusters"
    http://www.biomedcentral.com/1471-2105/14/252
    To construct this kernel, you must pass a list of kernels. the first kernel
    will be assumed to be the 'base' kernel, and will be computed everywhere.
    For every additional kernel, we assume another layer in the hierachy, with
    a corresponding column of the input matrix which indexes which function the
    data are in at that level.
    For more, see the ipython notebook documentation on Hierarchical
    covariances.
    """
    def __init__(self, kernels, name='hierarchy'):
        #assert all([k.input_dim==kernels[0].input_dim for k in kernels])
        assert len(kernels) > 1
        self.levels = len(kernels) -1
        input_max = max([k.input_dim for k in kernels])
        super(Hierarchical, self).__init__(kernels=kernels, extra_dims = range(input_max, input_max + len(kernels)-1), name=name)

    def K(self,X ,X2=None):
        K = self.parts[0].K(X, X2) # compute 'base' kern everywhere
        slices = [index_to_slices(X[:,i]) for i in self.extra_dims]
        if X2 is None:
            [[[np.add(K[s,s], k.K(X[s], None), K[s, s]) for s in slices_i] for slices_i in slices_k] for k, slices_k in zip(self.parts[1:], slices)]
        else:
            slices2 = [index_to_slices(X2[:,i]) for i in self.extra_dims]
            [[[np.add(K[s,ss], k.K(X[s], X2[ss]), K[s, ss]) for s,ss in zip(slices_i, slices_j)] for slices_i, slices_j in zip(slices_k1, slices_k2)] for k, slices_k1, slices_k2 in zip(self.parts[1:], slices, slices2)]
        return K

    def Kdiag(self,X):
        return np.diag(self.K(X))

    def gradients_X(self, dL_dK, X, X2=None):
        raise NotImplementedError

    def update_gradients_full(self,dL_dK,X,X2=None):
        slices = [index_to_slices(X[:,i]) for i in self.extra_dims]
        if X2 is None:
            self.parts[0].update_gradients_full(dL_dK, X, None)
            for k, slices_k in zip(self.parts[1:], slices):
                target = np.zeros(k.size)
                def collate_grads(dL, X, X2, target):
                    k.update_gradients_full(dL,X,X2)
                    target += k.gradient
                [[collate_grads(dL_dK[s,s], X[s], None, target) for s in slices_i] for slices_i in slices_k]
                k.gradient[:] = target
        else:
            raise NotImplementedError

In [ ]:
kfunc.active_dims, p, kreplicate.input_dim

In [ ]:
kfuncDesign = GPy.kern.Hierarchical([kfunc.copy(), kfunc.copy()])

kfuncDesign.input_dim, kfuncDesign._all_dims_active, kfuncDesign.rbf_1.variance 

In [ ]:
kreplicate

In [ ]:
kDesign = Hierarchical([kfuncDesign, kreplicate.copy()])

In [ ]:
kDesign.input_dim

In [ ]:
xdesign = np.zeros((2*x.shape[0]*y.shape[1], p + 2))

for i in range(p):
    xdesign[:,i] = np.tile(x[:,0]**(i+1),2*y.shape[1])
xdesign[xdesign.shape[0]/2:,-2] = 1
xdesign[:,-1] = np.repeat(range(y.shape[1]*2),x.shape[0])

plt.plot(xdesign)

In [ ]:
plt.imshow(kDesign.K(xdesign),interpolation='none')

In [ ]:
kfuncDesign.rbf_1.variance = .2*kfuncDesign.rbf.variance 

In [ ]:
samples = scipy.stats.multivariate_normal.rvs(np.zeros(xdesign.shape[0]), kDesign.K(xdesign))

In [ ]:
samples = samples.reshape((y.shape[1]*2,x.shape[0])).T

In [ ]:
plt.plot(samples);

In [ ]:
xother,yother,_,_ = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',Bio='F')
yother = (yother-yother.mean())/yother.std()


In [ ]:
plt.plot(xother,yother);

In [ ]:
k

In [ ]:
xtest = np.zeros((40,3))
xtest[:,0] = np.tile(np.linspace(-1,1,10),4)
xtest[:,1] = np.repeat(range(4),10)
xtest[20:,2] = 1

xtest

In [ ]:
# khier = GPy.kern.Hierarchical([GPy.kern.RBF(1),GPy.kern.RBF(1),GPy.kern.RBF(1)])
khier = GPy.kern.Hierarchical([GPy.kern.RBF(1),GPy.kern.RBF(1)])
# khier._all_dims_active = np.array([0,2])
# khier.active_dims = np.array([0,2])
# khier.extra_dims = np.array([2])
plt.imshow(khier.K(xtest),interpolation='none')

In [ ]:
khier2 = GPy.kern.Hierarchical([GPy.kern.RBF(1),GPy.kern.RBF(1)])
khier2._all_dims_active = np.array([0,2])

plt.imshow(khier2.K(xtest),interpolation='none')

In [ ]:
khierall = khier + khier2

plt.imshow(khierall.K(xtest),interpolation='none')

In [ ]:
plt.plot(scipy.stats.multivariate_normal.rvs(np.zeros(40),khierall.K(xtest)))